- Ggf. zu float16 konvertieren checken, ob finaler df mit time slices dann deutlich kleiner und performance testen
- column 'd' in training df löschen?
- paralletl computing einstellen
- use_multiprocessing in keras auf true setzen (model.fit agument)
- Cross validation?
- Ensemble learning?

In [2]:
#Import data handling libraries
import pandas as pd
import numpy as np

In [3]:
#Specify directories
#code_env = 'kaggle'
code_env = 'local'


if code_env=='local':
    ###local###
    #get parent folder of current directory
    parent_dir = '/Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction'

    #Directory resources
    res_dir = parent_dir + '/res/'
    src_dir = parent_dir + '/src/'
    prc_dir = src_dir + 'processed_data/' # Processed data directory with pickled dataframes

if code_env=='kaggle':
    ###On Kaggle###
    res_dir = '/kaggle/input/m5-forecasting-accuracy/'
    prc_dir = '/kaggle/input/processed-data-v2/'

In [4]:
# Create variables
BASE     = prc_dir +'df_1.pkl'
CALENDAR = prc_dir +'df_2.pkl'
NUM_ITEMS = 30490 # Number of items per each day

In [20]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam
from keras import backend as K

In [6]:
# Read in df_train_conv from pickle file
df_all_data = pd.concat([pd.read_pickle(BASE),
           pd.read_pickle(CALENDAR)], 
           axis=1)

In [7]:
# Define categorical and numerical columns
categorical_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'is_available',
                    'd', 'wday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
                    'snap_CA', 'snap_TX', 'snap_WI', 'mday', 'week', 'month', 'year']
numerical_cols = ['sell_price']

target_col = 'sales_amount'

# Convert categorical columns to category dtype and encode with cat.codes
for col in categorical_cols:
    df_all_data[col] = df_all_data[col].astype('category').cat.codes

# Normalize numerical columns
scaler = MinMaxScaler()
df_all_data[numerical_cols] = scaler.fit_transform(df_all_data[numerical_cols].astype(np.float32))
df_all_data[target_col] = scaler.fit_transform(df_all_data[[target_col]].astype(np.float32))

In [8]:
# Splitting the data in train, validation and test set; days are now 0 based, so have to shift by 1
TRAIN_END = 1913 - 28 # 1885 -> Train only until the 28 days before the end of the data
VAL_END = 1913

df_train = df_all_data[df_all_data['d'] < TRAIN_END]
df_val = df_all_data[(df_all_data['d'] >= TRAIN_END) & (df_all_data['d'] < VAL_END)]
df_test = df_all_data[df_all_data['d'] >= VAL_END]

del df_all_data

In [9]:
# Custom Generator Function
def lstm_data_generator(df, target, days_per_sequence=1, batch_size=32):
    total_sequences = (len(df) - NUM_ITEMS * days_per_sequence) // NUM_ITEMS # 1878 for train, 21 for val and test
    while True:  # Loop indefinitely, the model's fit method will break the loop
        for i in range(0, total_sequences, batch_size): # 0, 32, 64, ...1878
            batch_sequences = []
            batch_targets = []
            for b in range(batch_size): # 0, 1, 2,... 32
                if i + b < total_sequences: # 0, 0; 0, 1; 0, 2; ...; 0, 32; 32, 0; 32, 1; ...
                    start_idx = (i + b) * NUM_ITEMS
                    end_idx = start_idx + NUM_ITEMS * days_per_sequence
                    batch_sequences.append(df.iloc[start_idx:end_idx, :].drop(target, axis=1).to_numpy()) #drop target column, Only the values in the DataFrame will be returned, the axes labels will be removed.
                    batch_targets.append(df.iloc[end_idx:end_idx + NUM_ITEMS][target].to_numpy())
            yield np.array(batch_sequences), np.array(batch_targets)

In [10]:
# Usage
time_steps = 7  # Number of days per sequence
batch_size = 32  # Size of each batch
epochs= 2
train_gen = lstm_data_generator(df_train, target_col, time_steps, batch_size)
val_gen = lstm_data_generator(df_val, target_col, time_steps, batch_size)

In [11]:
# Custom RMSE loss function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [21]:
# Define the LSTM Model with the functional API to account for a more complex architecture
# input_shape = (time_steps, len(df_train.columns) - 1)
# model_input = Input(shape=input_shape)
# x = LSTM(50, activation='tanh')(model_input)
# x = Dense(1)(x)
# model = Model(inputs=model_input, outputs=x)

# model.compile(optimizer=Adam(), loss=rmse) # first placed solution used RMSE as loss function



# Assuming you have your LSTM model defined
input_shape = (time_steps, len(df_train.columns) - 1)
model = Sequential([
    LSTM(50, activation='tanh', input_shape=(input_shape)),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss=rmse)
# model possibly ergänzen um metrics=['accuracy']

2024-01-10 17:56:28.921923: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-10 17:56:28.922553: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-10 17:56:28.923210: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [18]:
# df head with all columns displayed
#pd.set_option('display.max_columns', None)
#df_train.head()

In [ ]:
# Train the model using the generator
# model.fit(x=train_gen, steps_per_epoch=(len(df_train) // (batch_size * NUM_ITEMS)), epochs=3) # x: In case of a generator the target y will be obtained from x; steps_per_epoch: 57mio // (32*30490) = 58

# Fit the model and store history for later evaluation
history = model.fit(
    x=train_gen,
    epochs=epochs,
    steps_per_epoch=(len(df_train) // (batch_size * NUM_ITEMS)),
    validation_data=val_gen,
    validation_steps=(len(df_train) // (batch_size * NUM_ITEMS))
)

In [ ]:
# Save the model to a specified directory
if code_env=='local':
    ###local###
    model.save('/Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction/models/V1_1layer_LSTM.h5')
    
if code_env=='kaggle':
    ###On Kaggle###
    model.save('/kaggle/working/models/V1_1layer_LSTM.h5')

In [15]:
# Start from here if you want to load the model
from keras.models import load_model

# Load the model from a specified directory
if code_env=='local':
    ###local###
    model = load_model('/Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction/src/models/V1_1layer_LSTM.h5', custom_objects={'rmse': rmse})

if code_env=='kaggle':
    ###On Kaggle###
    model = load_model('/kaggle/working/models/V1_1layer_LSTM.h5', custom_objects={'rmse': rmse})

2024-01-10 17:50:00.990982: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-10 17:50:00.991492: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-10 17:50:00.991955: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [29]:
import matplotlib.pyplot as plt
try:
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
except:
    print('No history to plot')

No history to plot


In [ ]:
# Now, let's define a function to calculate WRMSSE by calculating the RMSSE for each series and then multiplying by the weights and summing them up. 
def calculate_weights(sales_data, last_n_days=28):
    # sales_data: DataFrame with columns ['item_id', 'day', 'sales']
    # Sum sales for each item over the last_n_days
    item_sales = sales_data[sales_data['day'] > sales_data['day'].max() - last_n_days].groupby('item_id')['sales'].sum()
    # Total sales for all items
    total_sales = item_sales.sum()
    # Calculate weights
    weights = item_sales / total_sales
    return weights

def rmsse(y_true, y_pred, h, y_train):
    numerator = np.sum((y_true - y_pred) ** 2) / h
    denominator = np.sum(np.diff(y_train) ** 2) / (len(y_train) - 1) # np.diff to calc the diff for consecutive elements
    return np.sqrt(numerator / denominator)

def wrmsse(y_trues, y_preds, weights, h, y_trains):
    rmsse_values = [rmsse(y_true, y_pred, h, y_train) for y_true, y_pred, y_train in zip(y_trues, y_preds, y_trains)]
    return np.sum(np.array(weights) * np.array(rmsse_values))

In [ ]:
def evaluate_model_values(model, df_test, target_col, time_steps, batch_size, h, n):
    test_gen = lstm_data_generator(df_test, target_col, time_steps, batch_size)
    steps = max(1, len(df_test) // (batch_size * n))  # Ensure at least 1 step
    y_pred_normalized = model.predict(test_gen, steps=steps)
    y_pred_original = scaler.inverse_transform(y_pred_normalized)
    return y_pred_original

In [ ]:
evaluate_model_values(model, df_test, df_train, df_val, batch_size, time_steps, VAL_END)

In [ ]:
# Evaluate the model on the test set
def evaluate_model_wrmsse(model, df_test, df_train, df_val, batch_size, time_steps, n):
    test_gen = lstm_data_generator(df_test, target_col, time_steps, batch_size)
    steps = max(1, len(df_test) // (batch_size * n))  # Ensure at least 1 step
    y_pred_normalized = model.predict(test_gen, steps=steps)
    y_pred_original = scaler.inverse_transform(y_pred_normalized)
    y_true_normalized = df_test[target_col].values
    y_true_original = scaler.inverse_transform(y_true_normalized)
    
    #First concatenate all elements used for training (df_train and df_val)
    y_train_all_normalized = pd.concat([df_train[target_col], df_val[target_col]], axis=0).values
    y_train_all_original = scaler.inverse_transform(y_train_all_normalized)
    
    # Reshape the predictions and actuals to separate each item's time series
    y_pred_series = [y_pred_original[i::NUM_ITEMS] for i in range(NUM_ITEMS)]
    y_true_series = [y_true_original[i::NUM_ITEMS] for i in range(NUM_ITEMS)]

    # Similarly reshape the training data for RMSSE calculation
    y_train_all_series = [y_train_all_original[i::NUM_ITEMS] for i in range(NUM_ITEMS)]

    # Check - can be deleted later on
    print('len y_pred_series: ' + len(y_pred_series))
    print('len y_true_series: ' + len(y_true_series))
    print('len y_train_all_series: ' + len(y_train_all_series))
    
    # Calculate WRMSSE
    weights = calculate_weights(sales_data)
    wrmsse_score = wrmsse(y_trues=y_true_series, y_preds=y_pred_series, weights=weights, h=28, y_trains=y_train_all_series)

    print("Test WRMSSE: ", wrmsse_score)
    
    
    
    
    
    
    
    
    
    # Calculate wrmsse score
    wrmsse_score = wrmsse(
        y_trues=y_true_original,
        y_preds=y_pred_original,
        weights=calculate_weights(sales_data),
        h=28, # forecast horizon
        y_train=y_train_all_original
    )
    print("Test WRMSSE: ", wrmsse_score)

In [ ]:
# Call the evaluate function
evaluate_model_wrmsse(model, df_test, df_train, df_val, batch_size, time_steps, VAL_END)